# Deep Crossentropy method

In this section we'll extend your CEM implementation with neural networks! You will train a multi-layer neural network to solve simple continuous state space games. __Please make sure you're done with tabular crossentropy method from the previous notebook.__

![img](https://tip.duke.edu/independent_learning/greek/lesson/digging_deeper_final.jpg)



In [1]:
# In Google Colab, uncomment this:
# !wget https://bit.ly/2FMJP5K -O setup.py && bash setup.py

# XVFB will be launched if you run on a server
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY")) == 0:
    !bash ../xvfb start
    os.environ['DISPLAY'] = ':1'

../xvfb: line 2: $'\r': command not found
../xvfb: line 4: $'\r': command not found
../xvfb: line 8: $'\r': command not found
../xvfb: line 9: syntax error near unexpected token `$'in\r''
'


In [2]:
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Neural Network Policy

For this assignment we'll utilize the simplified neural network implementation from __[Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)__. Here's what you'll need:

* `agent.partial_fit(states, actions)` - make a single training pass over the data. Maximize the probabilitity of :actions: from :states:
* `agent.predict_proba(states)` - predict probabilities of all actions, a matrix of shape __[len(states), n_actions]__


### CEM steps
Deep CEM uses exactly the same strategy as the regular CEM, so you can copy your function code from previous notebook.

The only difference is that now each observation is not a number but a `float32` vector.

In [3]:
def select_elites(states_batch, actions_batch, rewards_batch, percentile=50):
    """
    Select states and actions from games that have rewards >= percentile
    :param states_batch: list of lists of states, states_batch[session_i][t]
    :param actions_batch: list of lists of actions, actions_batch[session_i][t]
    :param rewards_batch: list of rewards, rewards_batch[session_i]

    :returns: elite_states,elite_actions, both 1D lists of states and respective actions from elite sessions

    Please return elite states and actions in their original order 
    [i.e. sorted by session number and timestep within session]

    If you are confused, see examples below. Please don't assume that states are integers
    (they will become different later).
    """

    reward_threshold = np.percentile(rewards_batch, percentile)
    # print(f"reward_threshold = {reward_threshold}")
    ids = sorted([i for i, r in enumerate(rewards_batch) if r > reward_threshold])
    elite_states = [states_batch[i] for i in ids]
    elite_actions = [actions_batch[i] for i in ids]
    
    # print(f"elite_states = {elite_states}")
    # print(f"elite_actions = {elite_actions}")
    
    return elite_states, elite_actions

# Training loop
Generate sessions, select N best and fit to those.

In [4]:
from IPython.display import clear_output

def show_progress(rewards_batch, log, percentile, reward_range=[-990, +10]):
    """
    A convenience function that displays training progress. 
    No cool math here, just charts.
    """

    mean_reward = np.mean(rewards_batch)
    threshold = np.percentile(rewards_batch, percentile)
    log.append([mean_reward, threshold])

    clear_output(True)
    print("mean reward = %.3f, threshold=%.3f" % (mean_reward, threshold))
    plt.figure(figsize=[8, 4])
    plt.subplot(1, 2, 1)
    plt.plot(list(zip(*log))[0], label='Mean rewards')
    plt.plot(list(zip(*log))[1], label='Reward thresholds')
    plt.legend()
    plt.grid()

    plt.subplot(1, 2, 2)
    plt.hist(rewards_batch, range=reward_range)
    plt.vlines([np.percentile(rewards_batch, percentile)],
               [0], [100], label="percentile", color='red')
    plt.legend()
    plt.grid()

    plt.show()

# Results

## Assignment: MountainCar

By this moment you should have got enough score on [CartPole-v0](https://gym.openai.com/envs/CartPole-v0) to consider it solved (see the link). It's time to upload the result and get to something harder.

_if you have any trouble with CartPole-v0 and feel stuck, take a look at the forums_

Your assignment is to obtain average reward of __at least -150__ on `MountainCar-v0`.

See the tips section below, it's kinda important.
  
  
* Bonus quest: Devise a way to speed up training at least 2x against the default version
  * Obvious improvement: use [joblib](https://www.google.com/search?client=ubuntu&channel=fs&q=joblib&ie=utf-8&oe=utf-8)
  * Try re-using samples from 3-5 last iterations when computing threshold and training
  * Experiment with amount of training iterations and learning rate of the neural network (see params)
  
  
### Tips & tricks

* [Gym page for MountainCar](https://gym.openai.com/envs/MountainCar-v0)
* Sessions for MountainCar may last for 10k+ ticks. Make sure `t_max` parameter is at least 10k.
  * Also it may be a good idea to cut rewards via ">" and not ">=". If 90% of your sessions get reward of -10k and 20% are better, than if you use percentile 20% as threshold, R >= threshold __fails cut off bad sessions__ whule R > threshold works alright.
* _issue with gym_: Some versions of gym limit game time by 200 ticks. This will prevent cem training in most cases. Make sure your agent is able to play for the specified __t_max__, and if it isn't, try `env = gym.make("MountainCar-v0").env` or otherwise get rid of the `TimeLimit` wrapper.
* If it won't train it's a good idea to plot reward distribution and record sessions: they may give you some clue. If they don't, call course staff :)
* 20-neuron network is probably not enough, feel free to experiment.

You may find the following snippet useful:

In [5]:
# if you see "<classname> has no attribute .env", remove .env or update gym
env = gym.make("MountainCar-v0").env

# print(f"Initial state = {env.reset()}")
n_actions = env.action_space.n
state_dim = env.observation_space.shape[0]

print("state vector dim =", state_dim)
print("n_actions =", n_actions)

state vector dim = 2
n_actions = 3


In [6]:
from sklearn.neural_network import MLPClassifier

agent = MLPClassifier(
    hidden_layer_sizes=(25, 25),
    activation='tanh',
)

# initialize agent to the dimension of state space and number of actions
agent.partial_fit([env.reset()] * n_actions, range(n_actions), range(n_actions))

MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(25, 25), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [7]:
def generate_session(agent, t_max=1e4, i=None, quiet=False):
    """
    Play a single game using agent neural network.
    Terminate when game finishes or after :t_max: steps
    """
    
    # if not quiet:
        # if i is not None:
            # print(f" - Generating session {i}...")
        # else:
            # print(" - Generating session...")
    
    states, actions = [], []
    total_reward = 0

    s = env.reset()

    for t in range(int(round(t_max))):
        
        # use agent to predict a vector of action probabilities for state :s:
        probs = np.reshape(agent.predict_proba([s]), n_actions)

        assert probs.shape == (n_actions,), "make sure probabilities are a vector (hint: np.reshape)"
        
        # use the probabilities you predicted to pick an action
        # sample proportionally to the probabilities, don't just take the most likely action
        a = np.random.choice(
            a=range(n_actions),
            p=probs,
        )

        new_s, r, done, info = env.step(a)

        # record sessions like you did before
        states.append(s)
        actions.append(a)
        total_reward += r

        s = new_s
        if done:
            break
    return states, actions, total_reward

In [8]:
import multiprocessing as mp
from joblib import Memory

mem = Memory()
mem.cache(generate_session)

n_sessions = 100
percentile = 70
log = []
verbose = False

ImportError: No module named 'joblib'

In [ ]:
for i in range(100):
    # print(f"Generation {i}:")
    # generate new sessions
    pool = mp.Pool(processes=mp.cpu_count())
    # print(f" - Generated a pool of {mp.cpu_count()} workers.")
    sessions = pool.starmap(generate_session, [(agent, 2e4, j, not verbose) for j in range(n_sessions)])
    pool.close()
    pool.join()
    # print(f" - Simulated all {n_sessions} sessions for generation {i}.")
    
    # sessions = [generate_session(agent, t_max=1e4, i=i) for i in range(n_sessions)]
    # print(f"sessions[0] = {sessions[0]}")

    states_batch, actions_batch, rewards_batch = map(np.array, zip(*sessions))
#    if verbose:
#        print(f"states_batch[0:10] = {states_batch[0:10]}")
#        print(f"actions_batch[0:10] = {actions_batch[0:10]}")
#        print(f"rewards_batch[0:10] = {rewards_batch[0:10]}")

    elite_states, elite_actions = select_elites(states_batch, actions_batch, rewards_batch, percentile)

    # <partial_fit agent to predict elite_actions(y) from elite_states(X)>
    for i in range(len(elite_states)):
        agent.partial_fit(elite_states[i], elite_actions[i])

    show_progress(rewards_batch, log, percentile, reward_range=[np.min(rewards_batch), np.max(rewards_batch)])

    if np.mean(rewards_batch) > -150:
        # break
        print("You Win!")

### Submit to Coursera

In [ ]:
from submit import submit_mountain_car
submit_mountain_car(generate_session, agent, "lavoie.michel@gmail.com", "gQ1LBsI7rDIYfFi4")